In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os # to check if directory exists and create it if it doesn't
from datetime import datetime # to parse speech date
import spacy
import en_core_web_md
import csv
import nltk
import re
from sklearn.decomposition import PCA

In [5]:
# Load the two data sets needed
# string encodings to try utf_8, iso8859_15
facetPlotData = pd.read_csv('facetPlotData.csv')
df = pd.read_csv('fullEmotionData.csv', encoding='utf_8') # apostrophes aren't being read correctly
# Pandas reads date as string, cast to datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
facetPlotData['date'] = pd.to_datetime(facetPlotData['date'], format='%Y-%m-%d')
facetPlotData.shape # Should be (3000, 4) 

(3000, 4)

In [6]:
sources = ['text_oba', 'text_nyt', 'text_wsj']
for source in sources:
    tmp1 = [re.sub(r"[^\w\s^']", " ", text).lower() for text in df[source]]
    df[source] = [re.sub(' +', ' ', text).replace('\n', '').replace("â " , "'") for text in tmp1]

In [7]:
# Read in new articles from OtherSources directory
# Load up the files
path = './OtherSources/' # larger American Rhetoric dataset
list_of_files = []

# Get file names in target directory
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

# Open files and read in text
speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        text = f.read()
    f.close()
    speeches.append(text)

#clean out xa0 space characters
[speech.replace(u'\xa0', '') for speech in speeches]; # ; supresses output

info = [(file.split('_')[0][15:]+'.txt', file.split('_')[1][:-4]) for file in list_of_files]

# speeches[i] has the article text
# info[i][0] has the filename to match row of original Obama speech
# info[i][1] has the source name for naming the column
# Some new source/column names are: FoxNews, InfoWars, NationalReview, Intercept

for i in range(len(info)):
    df[info[i][1]] = None

for i in range(len(list_of_files)):
    df.loc[df['file'] == info[i][0], [info[i][1]]] = speeches[i]

<A HREf="https://arxiv.org/abs/1905.05583">Paper with some stuff on dealing with long text</A> This doesn't seem to be an issue with spacy encodings

In [8]:
nlp = spacy.load('en_core_web_md')

In [9]:
sources = ['text_oba', 'text_nyt', 'text_wsj', 'FoxNews', 'InfoWars', 'NationalReview', 'Intercept']
enc_col = ['enc_oba', 'enc_nyt', 'enc_wsj', 'enc_fn', 'enc_iw', 'enc_nr', 'enc_int']

# initialize the new encoding columns
for col in enc_col:
    df[col]=None

for i in range(len(df)):
    for j in range(len(sources)):
        #print(i, sources[j])
        if df[sources[j]][i] != None:
            # dfmi.loc[:, ('one', 'second')]
            df[enc_col[j]][i] = list(nlp(df[sources[j]][i]).vector)

C:\Users\peter\AppData\Local\Temp\ipykernel_20864\1934210802.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[enc_col[j]][i] = list(nlp(df[sources[j]][i]).vector)


In [16]:
pcaBiplotData = pd.DataFrame(columns=['date','source'])

In [85]:
# Reshape data for pca analysis and plotting
for i in range(len(df)):
    for j in range(len(sources)):
        if df.loc[i,sources[j]] != None:
            index = (i*len(sources))+j
            pcaBiplotData.loc[index, 'date' ] = pd.to_datetime(df['date'][i])
            pcaBiplotData.loc[index, 'source' ] = sources[j]
            for k in range(len(df['enc_oba'][99])):
                columnname = 'enc'+str(k+1)
                # initialize column, not sure this is necessary
                pcaBiplotData.loc[index, columnname] = None
                encoding = enc_col[j]
                pcaBiplotData.loc[index, columnname] = df[encoding][i][k]
pcaBiplotData = pcaBiplotData.reset_index()

In [88]:
# Selecting dataframe subset with columns that start with 'enc'
# pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')]

In [89]:
# All encoding values are already on same scale
pca = PCA(n_components = 2)
pcafit = pca.fit_transform(pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')])

In [93]:
#plot_data = pd.DataFrame()
#plot_data = pd.concat([pd.DataFrame(pcafit), pcaBiplotData['source']], axis=1, ignore_index=True)

<A HREF="https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express">Plotly discrete color palettes/sequences</A>

In [126]:
fig = px.scatter(pcafit, 
                 x=0, 
                 y=1, 
                 color=pcaBiplotData['source'],
                color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [128]:
pcaBiplotData.to_csv('pcaBiplotData.csv', index=False)

In [130]:
df.to_csv('df_encodings.csv')